In [34]:
import os
import pandas as pd

# Define the path to the folder
path = 'RMMS/'

# Get a list of all files in the folder
files = [f for f in os.listdir(path) if f.endswith('.lrps.htm')]

# Initialize an empty list to store DataFrames
dfs = []

# Loop through each file
for file in files:
    # Read HTML tables into a list of DataFrames
    tables = pd.read_html(os.path.join(path, file))
    
    # Combine all tables into a single DataFrame
    df = pd.concat(tables)
    
    # Remove the first row and rows above row 9
    df = df.iloc[9:].reset_index(drop=True)
    
    # Set the correct header using the values from row 9
    df.columns = df.iloc[0]
    
    # Drop the now redundant row 9
    df = df.iloc[1:].reset_index(drop=True)
    
    # Drop the first and last columns
    df = df.iloc[:, 1:-1]
    
    # Add a new column for road number
    road_number = file.split('.')[0]  # Extract road number from file name
    df['road_number'] = road_number
    
    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dfs, ignore_index=True)

# Save the combined DataFrame to a CSV file
combined_df.to_csv('combined_lrps.csv', index=False)


In [37]:
combined_df.shape
combined_df.to_csv('RMMS/combined_lrps.csv', index=False)

In [9]:
import pandas as pd

traffic_df = pd.read_csv('traffic.csv')
traffic_df.head()

,road,name,LRP,Offset,Chainage,LRP.1,Offset.1,Chainage.1,(Km),Heavy Truck,...,Car,Auto Rickshaw,Motor Cycle,Bi-Cycle,Cycle Rickshaw,Cart,Motorized,Non Motorized,Total AADT,(AADT)
0,N1,Jatrabari - Int.with Z1101 (Left) (Left),LRPS,0,0.000,LRPS,822,0.822,0.822,402.0,...,1851.0,2980.0,398.0,232.0,889.0,0.0,18236.0,1121.0,19357.0,19357.0
1,N1,Jatrabari - Int.with Z1101 (Left) (Right),LRPS,0,0.000,LRPS,822,0.822,0.822,660.0,...,2608.0,2508.0,436.0,213.0,1088.0,0.0,20236.0,1301.0,21537.0,21537.0
2,N1,Int.with Z1101 - Signboard (Left) R111 (Left),LRPS,822,0.822,LRPS,4175,4.175,3.353,660.0,...,2608.0,2508.0,436.0,213.0,1088.0,0.0,20236.0,1301.0,21537.0,21537.0
3,N1,Int.with Z1101 - Signboard (Left) R111 (Right),LRPS,822,0.822,LRPS,4175,4.175,3.353,402.0,...,1851.0,2980.0,398.0,232.0,889.0,0.0,18236.0,1121.0,19357.0,19357.0
4,N1,Signboard - Shimrail (Left)R110 (Left),LRPS,4175,4.175,LRPS,7181,7.181,3.006,91.0,...,1690.0,2266.0,1087.0,75.0,1198.0,0.0,16288.0,1273.0,17561.0,17561.0


In [12]:
traffic_df= traffic_df.rename(columns = { 'LRP':'LRP Start', 'Offset': 'Offset Start', 'Chainage':'Chainage Start', 'LRP.1':'LRP End', 'Offset.1': 'Offset End', 'Chainage.1':'Chainage End'})
traffic_df.head()

,road,name,LRP Start,Offset Start,Chainage Start,LRP End,Offset End,Chainage End,(Km),Heavy Truck,...,Car,Auto Rickshaw,Motor Cycle,Bi-Cycle,Cycle Rickshaw,Cart,Motorized,Non Motorized,Total AADT,(AADT)
0,N1,Jatrabari - Int.with Z1101 (Left) (Left),LRPS,0,0.000,LRPS,822,0.822,0.822,402.0,...,1851.0,2980.0,398.0,232.0,889.0,0.0,18236.0,1121.0,19357.0,19357.0
1,N1,Jatrabari - Int.with Z1101 (Left) (Right),LRPS,0,0.000,LRPS,822,0.822,0.822,660.0,...,2608.0,2508.0,436.0,213.0,1088.0,0.0,20236.0,1301.0,21537.0,21537.0
2,N1,Int.with Z1101 - Signboard (Left) R111 (Left),LRPS,822,0.822,LRPS,4175,4.175,3.353,660.0,...,2608.0,2508.0,436.0,213.0,1088.0,0.0,20236.0,1301.0,21537.0,21537.0
3,N1,Int.with Z1101 - Signboard (Left) R111 (Right),LRPS,822,0.822,LRPS,4175,4.175,3.353,402.0,...,1851.0,2980.0,398.0,232.0,889.0,0.0,18236.0,1121.0,19357.0,19357.0
4,N1,Signboard - Shimrail (Left)R110 (Left),LRPS,4175,4.175,LRPS,7181,7.181,3.006,91.0,...,1690.0,2266.0,1087.0,75.0,1198.0,0.0,16288.0,1273.0,17561.0,17561.0


In [21]:
# Gaat hier nu alleen op LRP nummer maar moet ook nog op weg nummer (bv N1 of Z8004), even uitzoeken hoe dit werkt. 
combined_lrps = pd.read_csv('combined_lrps.csv')

latlonload = pd.merge(traffic_df, combined_lrps, left_on=['LRP Start','road'], right_on=['LRP No', 'road_number'] )

In [23]:
latlonload.tail(20)

,road,name,LRP Start,Offset Start,Chainage Start,LRP End,Offset End,Chainage End,(Km),Heavy Truck,...,Non Motorized,Total AADT,(AADT),LRP No,Road Chainage,LRP TYPE,Description,Latitude Decimal,Longitued Decimal,road_number
6172,Z8915,Bagmara - Bangla Bazar (Int.with R890),LRPS,0,0.000,LRP004,1020,4.350,4.350,0.0,...,1148.0,3522.0,3522.0,LRPS,0,Others,Start at Bagmara Infront of Launch Ghat.,22.5893886,90.6194716,Z8915
6173,Z8915,Bagmara - Bangla Bazar (Int.with R890),LRPS,0,0.000,LRP004,1020,4.350,4.350,0.0,...,1148.0,3522.0,3522.0,LRPS,0,Others,Start at Bagmara Infront of Launch Ghat.,22.5893886,90.6194716,Z8915
6174,Z8915,Bangla Bazar (Int.with R890) - Int.with Z8916,LRP004,1020,4.350,LRP010,1150,10.470,6.120,0.0,...,776.0,3117.0,3117.0,LRP004,3.366,Km Post,Inf missing,22.5926941,90.6494719,Z8915
6175,Z8915,Bangla Bazar (Int.with R890) - Int.with Z8916,LRP004,1020,4.350,LRP010,1150,10.470,6.120,0.0,...,776.0,3117.0,3117.0,LRP004,3.366,Km Post,Inf missing,22.5926941,90.6494719,Z8915
6176,Z8915,Bangla Bazar (Int.with R890) - Int.with Z8916,LRP004,1020,4.350,LRP010,1150,10.470,6.120,0.0,...,776.0,3117.0,3117.0,LRP004,3.366,Km Post,Inf missing,22.5926941,90.6494719,Z8915
6177,Z8915,Int.with Z8916 - Daulatkhan,LRP010,1150,10.470,LRP014,2290,15.660,5.190,0.0,...,776.0,3117.0,3117.0,LRP010,9.376,Km Post,Inf missing,22.6011389,90.7048333,Z8915
6178,Z8915,Int.with Z8916 - Daulatkhan,LRP010,1150,10.470,LRP014,2290,15.660,5.190,0.0,...,776.0,3117.0,3117.0,LRP010,9.376,Km Post,Inf missing,22.6011389,90.7048333,Z8915
6179,Z8915,Int.with Z8916 - Daulatkhan,LRP010,1150,10.470,LRP014,2290,15.660,5.190,0.0,...,776.0,3117.0,3117.0,LRP010,9.376,Km Post,Inf missing,22.6011389,90.7048333,Z8915
6180,Z8916,Guingerhat (Int.with R890) - Int.with Z8915,LRPS,0,0.000,LRP004,3626,7.626,7.626,0.0,...,977.0,2440.0,2440.0,LRPS,0,Others,Start from R890 at Guingerhat Bazar Dhola,22.6254993,90.6617222,Z8916
6181,Z8916,Guingerhat (Int.with R890) - Int.with Z8915,LRPS,0,0.000,LRP004,3626,7.626,7.626,0.0,...,977.0,2440.0,2440.0,LRPS,0,Others,Start from R890 at Guingerhat Bazar Dhola,22.6254993,90.6617222,Z8916
